# Rascunho

In [2]:
import numpy as np
import pystan as ps
import sympy as sp

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

%matplotlib inline

import pickle

import sys
sys.path.append('../scripts/')

from utilits import ParameterAlpha, BivariateBeta

In [3]:
m1,m2,v1,v2,rho = sp.symbols('m1 m2 v1 v2 rho')
alpha3,alpha4 = sp.symbols('alpha_3 alpha_4')
alpha1 = (m1+m2-1)/(1-m1)*alpha3 + m2/(1 - m1)*alpha4
alpha2 = (1-m2)/(1-m1)*alpha3 + (m1-m2)/(1-m1)*alpha4 
tilde_a = alpha1+alpha2+alpha3+alpha4
corr = (alpha1*alpha4 - alpha2*alpha3)/(tilde_a**2*sp.sqrt((m1*m2*(1-m1)*(1-m2))))
corr = sp.simplify(corr)

In [4]:
sp.solve(rho - corr, alpha3)[0]

alpha_4*(-m1*m2 + m2 - rho*sqrt(m1*m2*(m1*m2 - m1 - m2 + 1)))/(m1*m2 - m1 - m2 + rho*sqrt(m1*m2*(m1*m2 - m1 - m2 + 1)) + 1)